<a href="https://colab.research.google.com/github/ducanh2505/scrape_data/blob/main/utilize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages

In [1]:
!pip install requests-html

     |████████████████████████████████| 153kB 6.9MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=7be51728782c996f72f8c2edc67774cc125e7c55b689789cffbc10ec884fb399
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24133 sha256=b3a10185530523e48bbb79fc64bc59c9a75615e5b00546aaf2d70505a38d759f
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
Successfully built fake-useragent parse
ERROR: pyppeteer 0.2.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


### import packages and define some variables

In [37]:

from requests_html import HTMLSession
import requests
import json
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np

In [3]:
session = HTMLSession()
home = 'https://gearvn.com'
url = 'https://gearvn.com/collections/laptop'

### get_cat_menu()


In [41]:
r = session.get(home)
len(np.unique([t.find('a',first=True).attrs['href'] for t in r.html.find('li.cat-menu-item')]))


15

In [45]:
def get_cat_menu():
  r = session.get(home)
  return np.unique([t.find('a',first=True).attrs['href'] for t in r.html.find('li.cat-menu-item')]).tolist()
cat_menu = get_cat_menu()
cat_menu

['/collections/ban-phim-co-gaming',
 '/collections/gaming-headphones',
 '/collections/gaming-mouse',
 '/collections/gaming-pc-gearvn',
 '/collections/ghe-choi-game',
 '/collections/laptop',
 '/collections/linh-kien-may-tinh',
 '/collections/man-hinh',
 '/collections/may-choi-game',
 '/collections/pc-van-phong',
 '/collections/pc-workstation',
 '/collections/phu-kien',
 '/collections/thiet-bi-mang',
 '/collections/thiet-bi-tai-nghe-loa-audio-chuyen-nghiep',
 '/collections/thiet-bi-van-phong']

### def get_from_page()
get all links from page

In [4]:
def get_from_page(url,session=session):  
  r = session.get(url)
  product_row = r.html.find('.product-row')
  return [home + link.find('a[href]',first=True).attrs['href']\
                            for link in product_row]
a = get_from_page(url)

In [5]:
len(a),a

(20,
 ['https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa',
  'https://gearvn.com/products/laptop-asus-vivobook-d509da-ej116t-new',
  'https://gearvn.com/products/laptop-asus-d409da-ek499t',
  'https://gearvn.com/products/laptop-hp-15s-fq1107tu-193q3pa',
  'https://gearvn.com/products/laptop-acer-aspire-3-a315-56-37dv',
  'https://gearvn.com/products/laptop-asus-x509ja-ej427t',
  'https://gearvn.com/products/laptop-asus-a412fa-ek1187t',
  'https://gearvn.com/products/laptop-asus-experbook-p1410cja-ek354t',
  'https://gearvn.com/products/laptop-acer-aspire-5-a514-54-39ku',
  'https://gearvn.com/products/laptop-asus-vivobook-flip-tm420ia-ec155t',
  'https://gearvn.com/products/laptop-asus-vivobook-a515ea-bq489t',
  'https://gearvn.com/products/laptop-asus-x509ja-ej021t',
  'https://gearvn.com/products/laptop-asus-x509jp-ej012t',
  'https://gearvn.com/products/laptop-acer-aspire-5-a514-53-38gf',
  'https://gearvn.com/products/laptop-asus-vivobook-x409ja-ek014t-silver',
  'htt

### get_from_site()
scrawl all link from the site

In [55]:
def get_product_from_site(url,par=None,session= session):
  finish = False
  page = 1
  all_links = []
  while not finish:
    t = get_from_page(url+'?page={}'.format(page),session)
    if (len(t) > 0 ):
      page += 1
      all_links.extend(t)
      if par :
        progess = {}
        progess['len'] = len(t)
       
    else: 
      finish = True
  return all_links



In [49]:
all_links =get_product_from_site(url)
len(all_links)


102

### get_all_product()


In [57]:
def get_all_product(cat_menu=cat_menu):
  product_links = []
  par = tqdm(cat_menu,total =len(cat_menu))
  progess = {}
  for category in par:
    t = get_product_from_site(home+category,par)    
    product_links.extend(t)
    progess['len'] = len(t)
    progess['total'] = len(product_links)
    par.set_postfix(progess)
    bar.update()
  return product_links


In [60]:
with open("product_links.txt",'w') as f:
  f.writelines(get_all_product())

### get_from_product()
scrawl from a product

In [ ]:
product = all_links[0]
product

'https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa'

In [ ]:
r = session.get('https://gearvn.com/products/laptop-gaming-asus-rog-zephyrus-m-gu502gv-az079t')

text = r.html.find('.product_parameters')[0].find('p')[0].text.split("\n")
text = [t.split(':') for t in text]
text


  

[['Nhà sản xuất\xa0', ' Asus'],
 ['Xuất xứ\xa0', ' Chính hãng'],
 ['Bảo hành\xa0', ' 24 Tháng'],
 ['Tình trạng\xa0', ' Mới 100%'],
 ['Màu sắc', '\xa0Black'],
 [''],
 [''],
 [''],
 ['Quà tặng', ''],
 [''],
 ['★\xa0Túi chống shock Gearvn\xa0trị giá\xa0139.000 VNĐ'],
 [''],
 [''],
 [''],
 ['Default Title'],
 ['36,990,000₫'],
 ['- +'],
 ['(Vui lòng chọn số lượng trước khi mua hàng)'],
 ['Giá Cũ', ''],
 ['46,990,000₫'],
 [''],
 ['Giá KM', '36,990,000₫'],
 ['Đặt hàng']]

In [22]:
def get_from_product(links):
  products = [] 
  bar = tqdm(all_links,total = len(all_links))
  for product in bar:
    r = session.get(product)
    product_dictionary = {}

    # get general atributes
    general_attributes = r.html.find('table>tbody',first=True).find('tr')
    for attr in general_attributes:   
      if len(attr.find('td')) == 2 :
        product_dictionary[attr.find('td')[0].text] = attr.find('td')[1].text 
                                              


    # get product parameter
    text = r.html.find('.product_parameters')[0].find('p')[0].text.split("\n")
    text = [t.split(':') for t in text]
    for i,t in enumerate(text):
      if t[0] == 'Giá Cũ':
        product_dictionary['Giá Cũ'] = text[i+1][0]      
      elif len(t) == 2:
        product_dictionary[t[0]] = t[1]


    products.append(product_dictionary)
  return products

In [18]:
product_dictionary

{'CPU': 'INTEL Core i5 9400F 6C6T 9M 2.9GHz',
 'Case': 'XIGMATEK GEMINI (Mini Tower)',
 'Cooling': 'DEEPCOOL GAMMAXX 400 RED',
 'Giá Cũ': '17,290,000₫',
 'Giá KM': '14,790,000₫',
 'HDD': 'SEAGATE Barracuda 1TB',
 'Mainboard': 'MSI MAG B365M Mortar LGA1151v2',
 'PSU': 'Corsair CV450 80 Plus Bronze',
 'RAM': 'KINGSTON HyperX Fury 2x8G 2666',
 'SSD': 'PATRIOT Burst 2.5 Sata III 240GB',
 'VGA': 'MSI GTX 1650 Ventus XS 4G OC GDDR5'}